In [3]:
# section 1 load all the necessary modules and packages
import glob
import time
import netCDF4 as nc4
import numpy as np
import pandas as pd
import xarray as xr
# not neccessary for the function but for visualziation
import matplotlib.pyplot as plt
import warnings
import sys
import os
import platform
import shutil


In [4]:
###########################
# replacing string function
###########################
def replace_string (file_in, file_out, string_old, string_replaced):
    with open(file_in, "r+") as text_file:
        texts = text_file.read()
        for i in np.arange(len(string_old)):
            texts = texts.replace(string_old[i], string_replaced[i])
    with open(file_out, "w") as text_file:
        text_file.write(texts)


In [8]:
###########################
# get the evaporation for lake victoria 
###########################
file_names = sorted(glob.glob('../mizuRoute/input/mizuRoute_mswep_v1_*_subset.nc'))

datasets = [xr.open_dataset(file_name) for file_name in file_names]

merged_dataset = xr.concat(datasets, dim='time')

# number of years
time_variable = merged_dataset['time']
years = time_variable.dt.year
years = np.array(years)
n_years = np.unique(years)
print(n_years)


merged_dataset_slice = merged_dataset.sel(lat=-1.00, lon=33.00, method='nearest')

# find the location index of Lake Victoria
idx = np.array(np.where(ds['ID']==7000016))
idx = idx[0]
    
# slice
evaporation = np.array(merger_dataset['evaporation'][:,idx])
average_evap = sum(evaporation)/n_years

# get the ratio for scale
other_evap = np.array([1550, 1350, 1130, 1520, 1450, 1500, 1370, 1590, 1470, 1475, 1590, 1595, 1400])

# creat the scale factors
scales = other_evap/average_evap


[1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993
 1994]


NameError: name 'll' is not defined

In [ ]:
# Case in which data is located
shutil.copy('../mizuRoute/lake_victoria_temp.submit',
            '../mizuRoute/lake_victoria.submit')

for scale in scales:
    
    #
    replace_string('../mizuRoute/setting/lake_victoria_temp.control',
                   '../mizuRoute/setting/lake_victoria'+f"{float_number:.4f}"+'.control',
                   ['case_temp', 'scale_factor_temp'],
                   ['case'+f"{float_number:.4f}", f"{float_number:.4f}"])
    
    #
    with open('../mizuRoute/lake_victoria.submit', "w") as file:
        line_to_write = 'srun route_runoff.cesm_coupling.exe ../mizuRoute/setting/lake_victoria'+f"{float_number:.4f}"+'.control'+' \n'
        file.write(line_to_write)

